# CH6. Blog 앱 확장 - Tag 달기

* 현재 기능   
: 테이블에 존재하는 포스트 리스트를 보여주고 특정 포스트의 내용을 열람하는 기능   

* 개발할 기능   
: 각 포스트마다 태그를 달 수 있는 기능과 태그를 달고 태그별로 포스트의 리스트를 보여주며 태그 클라우드를 만드는 방법 추가

## 1. 애플리케이션 설계

* 블로그의 각 포스트마다 태그 달기   
* 해태그 클릭 시 해당 태그를 소유한 모든 포스트의 리스트 열람   
* 태그를 모아서 보여주는 태그 클라우드 생성


> **화면 UI 설계**   
: 기존 포스트 상세화면 수정   
  태그 관련 2개의 신규 화면 추가
  

> **테이블 설계**   
: Post 테이블 (+ tags 필드 [TaggableManager / Blank, Null] )


> **URL 설계**   
: /blog/tag/ - TagCloudTV(TemplateView) - taggit_cloud.html   
  /blog/tag/tagname/ - TaggedObjectLV(ListView) - taggit_post_list.html
  

> **프로젝트 순서**   
>1. settings.py   
: taggit, taggit_templatetag2 애플리케이션 등록   
>2. models.py   
: tags 필드 추가   
>3. admin.py   
: 태그 관련 내용 추가   
>4. makemigrations / migrate   
>5. urls.py   
: URL 정의 추가   
>6. views.py   
: 뷰 로직 추가   
>7. templates 디렉터리   
: 템플릿 파일 1개 수정, 2개 추가

## 2. 애플리케이션 등록

: 사용할 태그 패키지 등록

```
mysite/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'bookmark.apps.BookmarkConfig',
    'blog.apps.BlogConfig',
    'taggit.apps.TaggitAppConfig',
    'taggit_templatetags2',
]

MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')

TAGGIT_CASE_INSENSITIVE = True
TAGGIT_LIMIT = 50
```

## 3. 모델 코딩

: Post 테이블에 tags 필드 추가

> **TaggableManager**   
> 1. Tag 테이블 생성   
> 2. 테이블을 ManyToMany로 묶어줌

* **1. models.py 수정**

```
from django.db import models
from django.urls import reverse
from taggit.managers import TaggableManager

class Post(models.Model) :
    # id 자동 생성
    title = models.CharField(verbose_name = 'TITLE', max_length = 50)
    # allow_unicode로 한글 처리
    slug = models.SlugField('SLUG', unique = True, allow_unicode = True, help_text = 'one word for title alias.')
    description = models.CharField('DESCRIPTION', max_length = 100, blank = True, help_text = 'simple description text')
    content = models.TextField('CONTENT')
    # 생성 시각 저장 (auto_now_add)
    create_dt = models.DateTimeField('CREATE DATE', auto_now_add = True)
    # 저장 시각 저장 (auto_now)
    modify_dt = models.DateTimeField('MODIFY DATE', auto_now = True)
    # 태그
    tags = TaggableManager(blank = True)
    
    # 하단 내용 동일
```

*  **2. admin.py 수정**

```
from django.contrib import admin
from blog.models import Post

@admin.register(Post)
# Admin 사이트에서 보여지는 모습 지정
class PostAdmin(admin.ModelAdmin) :
    # 보여지는 컬럼
    list_display = ('id', 'title', 'modify_dt', 'tag_list')
    # 필터 사이드바 지정
    list_filter = ('modify_dt',)
    # 검색박스 지정
    search_fields = ('title', 'content')
    # slug는 title 필드로 채워지도록 지정
    prepopulated_fields = {'slug' : ('title',)}
    
    # N:N 관계인 Post 테이블과 tags 테이블의 성능 고려
    # 한 번에 쿼리로 미리 가져오기 위함
    def get_queryset(self, request) :
        return super().get_queryset(request).prefetch_related('tags')
    
    # tag_list 항목 열람
    def tag_list(self, obj) :
        return ', '.join(o.name for o in obj.tags.all())
```



## 4. URL 추가


```
blog/urls.py

from django.urls import path, re_path
from blog import views

app_name = 'blog'

urlpatterns = [
    # 2가지 요청을 처리하는 PostLV
    path('', views.PostLV.as_view(), name = 'index'),
    path('post/', views.PostLV.as_view(), name = 'post_list'),
    re_path(r'^post/(?P<slug>[-\w]+)/$', views.PostDV.as_view(), name = 'post_detail'),
    path('archive/', views.PostAV.as_view(), name = 'post_archive'),
    path('archive/<int:year>/', views.PostYAV.as_view(), name = 'post_year_archive'),
    path('archive/<int:year>/<str:month>/', views.PostMAV.as_view(), name = 'post_month_archive'),
    path('archive/<int:year>/<str:month>/<int:day>/', views.PostDAV.as_view(), name = 'post_day_archive'),
    path('archive/today/', views.PostTAV.as_view(), name = 'post_today_archive'),
    # 추가
    path('tag/', views.TagCloudTV.as_view(), name = 'tag_cloud'),
    path('tag/<str:tag>/', views.TaggedObjectLV.aS_view(), name = 'tagged_object_list'),
]
```

## 5. 뷰 코딩하기

: urls.py에 추가한 2개의 view 추가하기 (TagCloudTV, TaggedObjectLV)

```
blog/views.py

class TagCloudTV(TemplateView) :
    # 템플릿 내에서 클라우드 기능 처리
    template_name = 'taggit/taggit_cloud.html'


class TaggedObjectLV(ListView) :
    template_name = 'taggit/taggit_post_list.html'
    model = Post

    def get_queryset(self) :
        return Post.objects.filter(tags__name = self.kwargs.get('tag'))

    # html에 넘겨줄 컨텍스트 변수 오버라이딩
    # 태그 네임을 넘겨줌
    def get_context_data(self, **kwargs) :
        context = super().get_context_data(**kwargs)
        context['tagname'] = self.kwargs['tag']
        return context
```

## 6. 템플릿 코딩하기

: post_detail 수정 후 models.py에서 정의한 taggit/taggit_cloud.html과 taggit/taggit_post_list.html 추가하기


**1. post_detail.html 수정**

```
blog/templates/blog/post_detail

# 수정
    <!-- 포스트 내용 보여주기 -->
    <div class = "body">
        {{ object.content | linebreaks }}
    </div>

# 추가
    <br>

    <!-- 태그 관련 -->
    <div>
        <!-- TAGS 텍스트와 아이콘 출력 -->
        <b>TAGS</b> <i class = "fas fa-tag"></i>
        <!-- 커스텀 태그 이용을 위하여 모듈 로딩 -->
        {% load taggit_templatetags2_tags %}
        <!-- 컨텍스트 변수로 넘겨받은 오브젝트에서 태그 추출 -->
        {% get_tags_for_object object as "tags" %}
        <!-- 태그 네임 출력 -->
        {% for tag in tags %}
        <!-- 출력된 태그 네임이 포함된 리스트 출력 -->
        <a href = "{% url 'blog:tagged_object_list' tag.name %}"> {{ tag.name }} </a>
        {% endfor %}
        &emsp;
        <!-- 태그 클라우드 버튼 생성 -->
        <a href = "{% url 'blog:tag_cloud' %}"> <span class = "btn btn-info btn-sm"> TagCloud </span></a>
    </div>

    {% endblock %}
```

**2. taggit/taggit_cloud.html 추가**

```
blog/templates/taggit/taggit_coud.html

<!DOCTYPE html>
<html>

<body>
    {% extends "base.html" %}

    {% block title %} Taggit Cloud {% endblock %}

    {% block extra-style %}
    <style type = "text/css">
        .tag-cloud {
            width : 40%;
            margin-left : 30px;
            text-align : center;
            padding : 5px;
            border : 1px solid orange;
            background-color : #ffc;
        }
        .tag-1 {font-size : 12px;}
        .tag-2 {font-size : 14px;}
        .tag-3 {font-size : 16px;}
        .tag-4 {font-size : 18px;}
        .tag-5 {font-size : 20px;}
        .tag-6 {font-size : 24px;{
    </style>
    {% endblock %}

    {% block content %}
        <h1> Blog Tag Cloud </h1>
        <br>

        <div class = "tag-cloud">
                {% load taggit_templatetags2_tags %}
                <!-- 모든 태그 추출 -->
                {% get_tagcloud as tags %}
                <!-- 추출된 태그 순회 -->
                {% for tag in tags %}
                <!-- 태그 이름, 태그 사용횟수 출력 -->  
                <!-- 태그 중요도 반올림하여 입력 -->
                <span class = "tag-{{tag.weight|floatformat:0}}">
                    <a href = "{% url 'blog:tagged_object_list' tag.name %}"> {{tag.name}} ({{tag.num_times}}) </a>
                </span>
                {% endfor %}
        </div>
    {% endblock %}

</body>

</html>
```

**3. taggit/taggit_post_list.html 추가**

```
blog/templates/taggit/taggit_post_list.html

<!DOCTYPE html>
<html>

<body>

    {% extends "base.html" %}

    {% block title %} Taggit Post List {% endblock %}

    {% block content %}
        <h1> Posts for tag - {{ tagname }} </h1>
        <br>

        {% for post in object_list %}
            <h2><a href = '{{ post.get_absolute_url }}'> {{ post.title }} </a></h2>
            {{ post.modify_dt|date:"N d, Y" }}
            <p> {{ post.description }} </p>
        {% endfor %}
    {% endblock %}

</body>
</html>
```

## 확인하기

http://localhost:8000

*입력한 포스트에 태그를 추가하여 데이터를 수정한 후 확인*